In [ ]:
from launch_pod import launch_webui, stop_and_terminate_pod, remove_safetensors, pod_benchmark, killall_pods
from templates import *
from multiprocessing import Process
import os, time

In [ ]:
API_KEY = 'YOUR KEY HERE'
file_tag = "whatever_you_want"

In [ ]:
models = {
    "TheBloke/wizardLM-7B-GPTQ": {"template": "ALPACA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A4000", "additional_cmd": ""},
    "TheBloke/Wizard-Vicuna-7B-Uncensored-GPTQ": {"template": "VICUNA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A4000", "additional_cmd": ""},
    "TheBloke/vicuna-7B-GPTQ-4bit-128g": {"template": "VICUNA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/guanaco-7B-GPTQ": {"template": "ALPACA/SHORT", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A4000", "additional_cmd": ""},
    "TheBloke/vicuna-13B-1.1-GPTQ-4bit-128g": {"template": "VICUNA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/wizardLM-13B-1.0-GPTQ": {"template": "ALPACA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/guanaco-13B-GPTQ": {"template": "ALPACA/SHORT", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/Manticore-13B-GPTQ": {"template": "ALPACA/VICUNA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "mindrage/Manticore-13B-Chat-Pyg-Guanaco-GPTQ-4bit-128g.no-act-order.safetensors": {"template": "ALPACA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ": {"template": "VICUNA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/VicUnlocked-30B-LoRA-GPTQ": {"template": "ALPACA/VICUNA", "groupsize": "", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/WizardLM-30B-Uncensored-GPTQ": {"template": "ALPACA/VICUNA", "groupsize": "", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ": {"template": "VICUNA", "groupsize": "", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/guanaco-33B-GPTQ": {"template": "ALPACA/SHORT", "groupsize": "", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/guanaco-65B-GPTQ": {"template": "ALPACA/SHORT", "groupsize": "", "wbits": "4", "GPU": "NVIDIA A100-SXM4-80GB", "additional_cmd": ""},
    "Aeala/VicUnlocked-alpaca-65b-4bit": {"template": "ALPACA", "groupsize": "128", "wbits": "4", "GPU": "2xNVIDIA A100-SXM4-80GB", "additional_cmd": ""},
}
additional_models = {
    "togethercomputer/RedPajama-INCITE-Instruct-3B-v1": {"template": "Q_A", "groupsize": "", "wbits": "", "GPU": "NVIDIA RTX A4000", "additional_cmd": ""},
    "togethercomputer/RedPajama-INCITE-7B-Instruct": {"template": "Q_A", "groupsize": "", "wbits": "", "GPU": "NVIDIA RTX A4000", "additional_cmd": ""},
    "TheBloke/falcon-7b-instruct-GPTQ": {"template": "FALCON", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A4000", "additional_cmd": ""},
    "anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g": {"template": "ALPACA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/Nous-Hermes-13B-GPTQ": {"template": "ALPACA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/wizard-vicuna-13B-GPTQ": {"template": "VICUNA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "GeorgiaTechResearchInstitute/starcoder-gpteacher-code-instruct": {"template": "ALPACA_INPUT", "groupsize": "", "wbits": "", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "sahil2801/instruct-codegen-16B": {"template": "ALPACA_PREFIX1", "groupsize": "", "wbits": "", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/OpenAssistant-SFT-7-Llama-30B-GPTQ": {"template": "OPENA", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/WizardLM-30B-GPTQ": {"template": "VICUNA", "groupsize": "", "wbits": "4", "GPU": "NVIDIA RTX A6000", "additional_cmd": ""},
    "TheBloke/falcon-40b-instruct-GPTQ": {"template": "FALCON", "groupsize": "128", "wbits": "4", "GPU": "NVIDIA A100-SXM4-80GB", "additional_cmd": ""},
}

combined_models = {**models, **additional_models}

template_map = {"ALPACA" : ALPACA_TEMPLATE, 
                "VICUNA" : VICUNA_TEMPLATE,
                "ALPACA_INPUT" : ALPACA_INPUT_TEMPLATE,
                "OPENA" : OPENASSISTANT_TEMPLATE,
                "FALCON" : FALCON_TEMPLATE,
                "Q_A" : QA_TEMPLATE,
                "ALPACA_PREFIX1" : ALPACA_PREFIX_TEMPLATE,
                "ALPACA_MEDIUM" : ALPACA_TEMPLATE_MEDIUM, 
                "ALPACA_INPUT_MEDIUM" : ALPACA_INPUT_TEMPLATE_MEDIUM,
                "ALPACA_PREFIX1_MEDIUM" : ALPACA_PREFIX_TEMPLATE_MEDIUM,
                "SHORT" : SHORT_TEMPLATE,
                "VERYSHORT" : VERYSHORT_TEMPLATE
               }

In [ ]:
import os
import sys
import contextlib

def process_model(model, file_tag=""):
    model_file = model.replace("/", "_")
    pod_tag = model_file

    if not os.path.exists("logs"):
        os.mkdir("logs")

    log_filename = f"logs/log_{model_file}_{file_tag}.txt"  # Define your log file name here
    print("Starting", model)
    # Redirect standard output and error to the log file
    with open(log_filename, 'w') as f, contextlib.redirect_stdout(f), contextlib.redirect_stderr(f):
        gpu_type = combined_models[model]["GPU"]
        groupsize = combined_models[model]["groupsize"]
        wbits = combined_models[model]["wbits"]
        gptq_params = ""

        if groupsize:
            gptq_params += f"--groupsize {groupsize} "
        if wbits:
            gptq_params += f"--wbits {wbits} "

        templ = combined_models[model]["template"]
        if "/" in templ:
            templ = templ.split("/")[0]
        prompt = template_map[templ]
        print(pod_tag, gpu_type, gptq_params,"*** PROMPT ***", prompt,"*** *** ***", sep="\n")

        api_url, model_file, pod_id = launch_webui(api_key=API_KEY, 
                               model=model, 
                               gptq_params=gptq_params, 
                               gpu_type=gpu_type,
                               pod_num=pod_tag)


        print("starting benchmark...")
        pod_benchmark(model_file + "_" + templ + "_" + file_tag, prompt, assistant_tag=prompt[-15:], pod_id=pod_id, api_key=API_KEY, start_from=0, host=api_url, port=443, insert_func_stub=True)

        stop_and_terminate_pod(pod_id, API_KEY)

        # Close the log files
        sys.stdout.close()
        sys.stderr.close()
    print("Finished", model)


In [ ]:
# to_run = [f for f in sorted(combined_models.keys()) if "7B" in f][:4]
to_run = combined_models.keys()

In [ ]:
import concurrent.futures
import contextlib
import os

MAX_WORKERS = 8

from multiprocessing import Pool

def worker(model):
    return process_model(model, file_tag)

pool = Pool(processes=MAX_WORKERS)

for model in to_run:
    pool.apply_async(worker, (model,))

pool.close()
pool.join()

In [ ]:
killall_pods(API_KEY)